In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# recent https://forecast.weather.gov/product.php?site=MFL&issuedby=MIA&product=CLI&format=CI&version=2&glossary=0
# Old https://climatecenter.fsu.edu/climate-data-access-tools/downloadable-data


df = pd.read_csv("True values/miami_weather_complete.csv")
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [6]:
df = df.drop(columns = {"Unnamed: 0", "precipitation",	"mean temp"})
df

,actual_max_temp,min temp,Datetime
0,81.0,63.0,2000-01-01
1,79.0,66.0,2000-01-02
2,80.0,64.0,2000-01-03
3,80.0,71.0,2000-01-04
4,80.0,67.0,2000-01-05
...,...,...,...
9089,72.0,55.0,2024-11-23
9090,78.0,58.0,2024-11-24
9091,82.0,64.0,2024-11-25
9092,84.0,67.0,2024-11-26


In [14]:
openmeteo = pd.read_csv('Open Meteo forecast/MiamiOpenMeteoData.csv')

In [15]:
openmeteo['time'] = pd.to_datetime(openmeteo['time'], format='%m/%d/%y')

# Convert the datetime column back to the desired string format 'DD/MM/YYYY'
openmeteo['time'] = openmeteo['time'].dt.strftime('%m-%d-%Y')

openmeteo = openmeteo.rename(columns = {"time": "Datetime"})
openmeteo['Datetime'] = pd.to_datetime(openmeteo['Datetime'])
openmeteo.dtypes

Datetime                           datetime64[ns]
weather_code (wmo code)                     int64
temperature_2m_max (°F)                   float64
temperature_2m_min (°F)                   float64
temperature_2m_mean (°F)                  float64
apparent_temperature_max (°F)             float64
apparent_temperature_min (°F)             float64
apparent_temperature_mean (°F)            float64
sunrise (iso8601)                          object
sunset (iso8601)                           object
daylight_duration (s)                     float64
sunshine_duration (s)                     float64
precipitation_sum (mm)                    float64
rain_sum (mm)                             float64
snowfall_sum (cm)                         float64
precipitation_hours (h)                     int64
wind_speed_10m_max (mp/h)                 float64
wind_gusts_10m_max (mp/h)                 float64
wind_direction_10m_dominant (°)           float64
shortwave_radiation_sum (MJ/m²)           float64


In [40]:
newdf = pd.merge(df,openmeteo, how = "inner", on = "Datetime")

In [41]:
newdf = newdf.drop(columns = {'sunrise (iso8601)','sunset (iso8601)', 'weather_code (wmo code)'})

In [42]:
newdf['precipitation_hours (h)'] = newdf['precipitation_hours (h)'].astype(float)
newdf = newdf.set_index("Datetime")

In [43]:
# newdf = newdf.dropna()

In [44]:
newdf = newdf.drop(columns = {"temperature_2m_max (°F)", "temperature_2m_min (°F)"})
correlation_matrix = newdf.corr()
correlation_matrix['actual_max_temp']



,actual_max_temp,min temp,temperature_2m_mean (°F),apparent_temperature_max (°F),apparent_temperature_min (°F),apparent_temperature_mean (°F),daylight_duration (s),sunshine_duration (s),precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),precipitation_hours (h),wind_speed_10m_max (mp/h),wind_gusts_10m_max (mp/h),wind_direction_10m_dominant (°),shortwave_radiation_sum (MJ/m²),et0_fao_evapotranspiration (mm)
Datetime,,,,,,,,,,,,,,,,,
2000-01-01,81.0,63.0,72.3,75.8,70.5,72.7,38012.65,34230.95,0.1,0.1,0.0,1.0,13.5,21.3,80.0,13.40,3.20
2000-01-02,79.0,66.0,72.6,75.0,69.3,71.7,38036.15,34630.02,0.0,0.0,0.0,0.0,13.7,22.1,69.0,15.40,3.90
2000-01-03,80.0,64.0,73.7,75.8,71.3,73.9,38061.85,33985.65,0.5,0.5,0.0,3.0,15.5,24.6,101.0,11.78,3.16
2000-01-04,80.0,71.0,74.3,80.4,74.3,77.1,38089.70,34169.00,0.2,0.2,0.0,1.0,13.0,21.3,141.0,13.56,2.89
2000-01-05,80.0,67.0,71.1,76.1,69.8,73.1,38119.63,29369.16,0.6,0.6,0.0,6.0,15.7,26.6,28.0,10.42,2.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-23,72.0,55.0,63.7,68.9,52.1,59.5,38677.84,35480.02,0.0,0.0,0.0,0.0,8.2,20.8,329.0,17.32,3.81
2024-11-24,78.0,58.0,67.3,76.7,53.1,65.2,38625.05,35398.84,0.0,0.0,0.0,0.0,7.6,19.9,15.0,15.49,3.64
2024-11-25,82.0,64.0,71.9,80.5,64.6,73.1,38574.13,35353.12,0.0,0.0,0.0,0.0,9.8,25.1,45.0,13.38,3.07


In [1]:
# for i in newdf.columns:
#     plt.scatter(newdf['actual_max_temp'], newdf[i])
#     plt.xlabel('actual_max_temp')
#     plt.ylabel(i)
#     plt.show()